In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import zipfile
import matplotlib.pyplot as plt

from nltk.corpus import stopwords

import transformers
from transformers import AdamW, get_linear_schedule_with_warmup

import tokenizers
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve, auc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from tqdm import tqdm
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_table = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test_table = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
test_tags = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip')
train_table.describe()

In [ ]:
def multiply6(x):
    return 6*x
train_losses = [0.1950, 0.0043, 0.0164, 0.238]
train_accs = list(map(multiply6,[0.153, 0.16, 0.165, 0.166]))
valid_losses = [0.0588, 0.0961, 0.4839, 0.0004]
valid_accs = list(map(multiply6,[0.161, 0.16, 0.160, 0.160]))

x = np.arange(4)
fig, ax = plt.subplots(1, 2, figsize = (15,4))
ax[0].plot(x, train_losses)
ax[0].plot(x, valid_losses)
ax[0].set_ylabel('Losses', weight = 'bold')
ax[0].set_xlabel('Epochs')
ax[0].grid(alpha = 0.3)
ax[0].legend(labels = ['train losses', 'valid losses'])

ax[1].plot(x, train_accs)
ax[1].plot(x, valid_accs)
ax[1].set_ylabel('Accuracy', weight = 'bold')
ax[1].set_xlabel('Epochs')
ax[1].legend(labels = ['train acc', 'valid acc'])

ax[1].grid(alpha = 0.3)
fig.suptitle('Fold = '+str(2), weight = 'bold') 

In [ ]:
train_losses = [0.0112, 0.1611, 0.0021, 0.0003]
train_accs = list(map(multiply6,[0.155, 0.162, 0.165, 0.166]))
valid_losses = [0.1226, 0.1395, 0.0290, 0.4823]
valid_accs = list(map(multiply6,[0.160, 0.161, 0.161, 0.161]))

x = np.arange(4)
fig, ax = plt.subplots(1, 2, figsize = (15,4))
ax[0].plot(x, train_losses)
ax[0].plot(x, valid_losses)
ax[0].set_ylabel('Losses', weight = 'bold')
ax[0].set_xlabel('Epochs')
ax[0].grid(alpha = 0.3)
ax[0].legend(labels = ['train losses', 'valid losses'])

ax[1].plot(x, train_accs)
ax[1].plot(x, valid_accs)
ax[1].set_ylabel('Accuracy', weight = 'bold')
ax[1].set_xlabel('Epochs')
ax[1].legend(labels = ['train acc', 'valid acc'])

ax[1].grid(alpha = 0.3)
fig.suptitle('Fold = '+str(4), weight = 'bold') 

In [ ]:
train_losses = [0.1716, 0.0130, 0.0145, 0.0031]
train_accs = list(map(multiply6,[0.157, 0.163, 0.165, 0.166]))
valid_losses = [0.1060, 0.0459, 0.0009, 0.0004]
valid_accs = list(map(multiply6,[0.161, 0.161, 0.160, 0.161]))

x = np.arange(4)
fig, ax = plt.subplots(1, 2, figsize = (15,4))
ax[0].plot(x, train_losses)
ax[0].plot(x, valid_losses)
ax[0].set_ylabel('Losses', weight = 'bold')
ax[0].set_xlabel('Epochs')
ax[0].grid(alpha = 0.3)
ax[0].legend(labels = ['train losses', 'valid losses'])

ax[1].plot(x, train_accs)
ax[1].plot(x, valid_accs)
ax[1].set_ylabel('Accuracy', weight = 'bold')
ax[1].set_xlabel('Epochs')
ax[1].legend(labels = ['train acc', 'valid acc'])

ax[1].grid(alpha = 0.3)
fig.suptitle('Fold = '+str(0), weight = 'bold') 

In [ ]:
train_losses = [0.0093, 0.4409, 0.0177, 0.0007]
train_accs = list(map(multiply6,[0.157, 0.163, 0.165, 0.166]))
valid_losses = [0.494, 0.0822, 0.0031, 0.0002]
valid_accs = list(map(multiply6,[0.161, 0.161, 0.161, 0.160]))

x = np.arange(4)
fig, ax = plt.subplots(1, 2, figsize = (15,4))
ax[0].plot(x, train_losses)
ax[0].plot(x, valid_losses)
ax[0].set_ylabel('Losses', weight = 'bold')
ax[0].set_xlabel('Epochs')
ax[0].grid(alpha = 0.3)
ax[0].legend(labels = ['train losses', 'valid losses'])

ax[1].plot(x, train_accs)
ax[1].plot(x, valid_accs)
ax[1].set_ylabel('Accuracy', weight = 'bold')
ax[1].set_xlabel('Epochs')
ax[1].legend(labels = ['train acc', 'valid acc'])

ax[1].grid(alpha = 0.3)
fig.suptitle('Fold = '+str(1), weight = 'bold') 

In [ ]:
train_losses = [0.0550, 0.0702, 0.0010, 0.0006]
train_accs = list(map(multiply6,[0.153, 0.163, 0.165, 0.166]))
valid_losses = [0.0782, 0.143, 0.0228, 0.0023]
valid_accs = list(map(multiply6,[0.161, 0.161, 0.161, 0.161]))

x = np.arange(4)
fig, ax = plt.subplots(1, 2, figsize = (15,4))
ax[0].plot(x, train_losses)
ax[0].plot(x, valid_losses)
ax[0].set_ylabel('Losses', weight = 'bold')
ax[0].set_xlabel('Epochs')
ax[0].grid(alpha = 0.3)
ax[0].legend(labels = ['train losses', 'valid losses'])

ax[1].plot(x, train_accs)
ax[1].plot(x, valid_accs)
ax[1].set_ylabel('Accuracy', weight = 'bold')
ax[1].set_xlabel('Epochs')
ax[1].legend(labels = ['train acc', 'valid acc'])

ax[1].grid(alpha = 0.3)
fig.suptitle('Fold = '+str(3), weight = 'bold') 

In [ ]:
# train_table = train_table.iloc[0:30000,:]
# train_table.head()

In [ ]:
import re

def text_preprocessing(text):
    #lower case
    text = text.lower()
    
    #pattern = [zero or more character]
    text = re.sub('\[.*?\]', '', text)
    
    #pattern = (zero or more character)
    text = re.sub('\(.*?\)', '', text)
    
    #pattern = with or without(http),://, one or more non-white space character, OR www, .,one or more non-white space character
    text = re.sub('https?://\S+|www\.\S+', '', text)
    
    #pattern = @some characters + space
    text = re.sub(r'(@.*?)[\s]', ' ', text) 
    
#     #pattern = num
#     text = re.sub(r'[0-9]+' , '' ,text)
    
    #pattern = space+@+A-Za-z0-9_
    text = re.sub(r'\s([@][\w_-]+)', '', text).strip()
    
    #pattern = &amp
    text = re.sub(r'&amp;', '&', text)
    
    #pattern = multiple space
    text = re.sub(r'\s+', ' ', text).strip()
    
    #replace #
    text = text.replace("#" , " ")
    
    #pattern = any new line
    text = re.sub('\n', '', text)
    
    encoded_string = text.encode("ascii", "ignore")
    decode_string = encoded_string.decode()
    return decode_string

In [ ]:
# train_table['processed_text'] = train_table['comment_text'].apply(str).apply(lambda x: text_preprocessing(x))
# train_table['easy_label'] = np.where(((train_table["toxic"] == 1) | (train_table["severe_toxic"] == 1) | (train_table["obscene"] == 1) | (train_table["threat"] == 1) | (train_table["insult"] == 1) | (train_table["identity_hate"] == 1)), 1, 0)
# nb_data = train_table[["processed_text", "easy_label"]]                                                                 
# Kfold = 5
# nb_data['kfold'] = nb_data.index % Kfold
# nb_data.describe()

In [ ]:
train_table['processed_text'] = train_table['comment_text'].apply(str).apply(lambda x: text_preprocessing(x))
train_table['easy_label'] = np.where(((train_table["toxic"] == 1) | (train_table["severe_toxic"] == 1) | (train_table["obscene"] == 1) | (train_table["threat"] == 1) | (train_table["insult"] == 1) | (train_table["identity_hate"] == 1)), 1, 0)
nb_data = train_table[['processed_text','toxic','severe_toxic','obscene','threat','insult','identity_hate']]
Kfold = 3
nb_data['kfold'] = nb_data.index % Kfold

In [ ]:
# test_table['processed_text'] = test_table['comment_text'].apply(str).apply(lambda x: text_preprocessing(x))
# test_table['easy_label'] = np.where(((test_table["toxic"] == 1) | (test_table["severe_toxic"] == 1) | (test_table["obscene"] == 1) | (test_table["threat"] == 1) | (test_table["insult"] == 1) | (test_table["identity_hate"] == 1)), 1, 0)
# test = test_table[["processed_text", "easy_label"]]  
# Kfold = 5
# test['kfold'] = test.index % Kfold

In [ ]:
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# max_len = 350
max_len = 128

class BertDataSet(Dataset):
    
    def __init__(self, sentences, toxic_labels):
        self.sentences = sentences
        #target is a matrix with shape [#1 x #6(toxic, obscene, etc)]
        self.targets = toxic_labels.to_numpy()
    
    def __len__(self):
        return len(self.sentences)
    
    
    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        bert_senten = tokenizer.encode_plus(
                                            text = sentence, 
                                            add_special_tokens = True, # Add `[CLS]` and `[SEP]`
                                            max_length = max_len, # Max length to truncate/pad
                                            pad_to_max_length=True, # Pad sentence to max length
                                            truncation = True, # truncate long sentence
                                            return_attention_mask = True  # Return attention mask
                                             )
        ids = torch.tensor(bert_senten['input_ids'], dtype = torch.long)
        mask = torch.tensor(bert_senten['attention_mask'], dtype = torch.long)
        toxic_label = torch.tensor(self.targets[idx], dtype = torch.float)
        
        
        return {
            'ids' : ids,
            'mask' : mask,
            'toxic_label':toxic_label
        }

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
import torch
from sklearn.metrics import roc_auc_score, plot_roc_curve, RocCurveDisplay, ConfusionMatrixDisplay

fold = 5 
for i in range(1):
    model = transformers.BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels = 6)
    checkpoint = torch.load("../input/toxic-comment/model%d.pth"%(i))
    model.load_state_dict(checkpoint['state_dict'])
    model.to(device)
    
    idxs = nb_data['kfold'] == i
    validation_y = nb_data[idxs][['toxic','severe_toxic','obscene','threat','insult','identity_hate']].reset_index(drop=True)
    validation_x = nb_data[idxs]["processed_text"].reset_index(drop=True)
    
    valid_dataset = BertDataSet(validation_x, validation_y)
    valid_dataloader = DataLoader(valid_dataset, batch_size = 16, shuffle = False, num_workers = 2, pin_memory = True)
    
    model.eval()
    correct_predictions = 0
    all_output_probs = []
    result = []
    num = 0
    for a in tqdm(valid_dataloader):
        num+=1
        losses = []
        ids = a['ids'].to(device, non_blocking = True)
        mask = a['mask'].to(device, non_blocking = True)
        output = model(ids, mask)
        output = output['logits'].squeeze(-1).to(torch.float32)
        output_probs = torch.sigmoid(output)
        preds = torch.where(output_probs.any() > 0.5, 1, 0)
        result.extend(output_probs.detach().cpu().numpy())
    

#     ConfusionMatrixDisplay.from_predictions(validation_y.to_numpy(), result)

In [ ]:
import torch.nn as nn

loss_fn = nn.BCEWithLogitsLoss()
loss_fn.to(device)
toxic_label = a['toxic_label'].to(device, non_blocking = True)
loss = loss_fn(output, toxic_label)     
loss

In [ ]:
test_result = list(map(lambda x: 1 if ((x>0.5).any()) else 0, result))

In [ ]:
from sklearn.metrics import roc_auc_score, RocCurveDisplay, roc_curve

roc_auc_score(validation_y.to_numpy(), result)
fpr, tpr, thresholds = roc_curve(validation_y.to_numpy(), result)
plt.plot(fpr, tpr)

In [ ]:
test_y = np.asarray(validation_y).flatten()

In [ ]:
pred_y = np.asarray(result).flatten()
pred_y

In [ ]:
from sklearn.metrics import RocCurveDisplay

ax = RocCurveDisplay.from_predictions(test_y, pred_y)

In [ ]:
preds_y = np.where((pred_y>0.5),1,0)

In [ ]:
font = {'family' : 'normal',
    'weight' : 'bold',
    'size'   : 22}
plt.rc('font', **font)

ConfusionMatrixDisplay.from_predictions(test_y, preds_y)

In [ ]:
train_table.head()

In [ ]:
train_table[nb_data['kfold']==0]['easy_label']

In [ ]:
# ConfusionMatrixDisplay.from_predictions(validation_y.to_numpy(), np.where(np.array(result)>0.5,1,0))
ConfusionMatrixDisplay.from_predictions(train_table[nb_data['kfold']==0]['easy_label'], test_result)

In [ ]:
roc_auc_score(train_table[nb_data['kfold']==0]['easy_label'], np.array(result)[:,0])

In [ ]:
tags = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
for i, item in enumerate(tags):
    print(roc_auc_score(train_table[nb_data['kfold']==1][item], np.array(result)[:,i]))
    ConfusionMatrixDisplay.from_predictions(train_table[nb_data['kfold']==1][item], list(map(lambda x: 1 if x>0.5 else 0, np.array(result)[:,i])))